In [1]:
%reload_ext autoreload
%autoreload 1
import joblib
import numpy as np
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report
import auxiliary.lists as aux_lists
import auxiliary.transformers as tr
import auxiliary.eda_functions as eda
import auxiliary.statistics as st
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from IPython.display import Markdown,display
import matplotlib.ticker as ticker
from tabulate import tabulate
%aimport auxiliary.lists
%aimport auxiliary.transformers
%aimport auxiliary.eda_functions
%aimport auxiliary.statistics

In [2]:
application_train = pl.read_csv("data/application_train.csv")
applicattion_test = pl.read_csv("data/application_test.csv")
bureau_balance = pl.read_csv("data/bureau_balance.csv")
bureau = pl.read_csv("data/bureau.csv")
installments_payments = pl.read_csv("data/installments_payments.csv")
previous_application = pl.read_csv("data/previous_application.csv")
credit_card_balance = pl.read_csv("data/credit_card_balance.csv")

In [3]:
applicattion_test = applicattion_test.with_columns(
    pl.lit(None).cast(pl.Int64).alias("TARGET")
)
applicattion_test = applicattion_test[
    [applicattion_test.columns[0]]
    + [applicattion_test.columns[-1]]
    + applicattion_test.columns[1:-1]
]
application_full = pl.concat([application_train, applicattion_test])

In [4]:
active_credit_cards = credit_card_balance.filter(
    (pl.col("MONTHS_BALANCE") == -1) & (pl.col("NAME_CONTRACT_STATUS") == "Active")
)

active_credit_cards = active_credit_cards.with_columns(
    pl.when((pl.col("AMT_CREDIT_LIMIT_ACTUAL") - pl.col("AMT_BALANCE")) < 0)
    .then(pl.lit(1, pl.Int16))
    .otherwise(pl.lit(0, pl.Int16))
    .alias("IS_OVER_LIMIT")
)

active_credit_cards = active_credit_cards.drop(
    columns=["MONTHS_BALANCE", "NAME_CONTRACT_STATUS"]
)

### Historic Behavior

In [5]:
historical_credit_card = credit_card_balance.filter(pl.col("MONTHS_BALANCE") != -1)
historical_credit_card = historical_credit_card.with_columns(
    pl.when((pl.col("AMT_CREDIT_LIMIT_ACTUAL") - pl.col("AMT_BALANCE")) < 0)
    .then(pl.lit(1, pl.Int16))
    .otherwise(pl.lit(0, pl.Int16))
    .alias("IS_OVER_LIMIT")
)

historical_credit_card = historical_credit_card.with_columns(
    (pl.col("AMT_BALANCE") - pl.col("AMT_CREDIT_LIMIT_ACTUAL")).alias("OVER_LIMIT")
)
historical_credit_card = historical_credit_card.with_columns(
    pl.when(pl.col("OVER_LIMIT") > 0)
    .then(pl.col("OVER_LIMIT"))
    .otherwise(pl.lit(0))
    .alias("OVER_LIMIT")
)

historical_credit_card_agg = pl.DataFrame(active_credit_cards["SK_ID_PREV"])

History length:

In [6]:
historical_credit_card_agg = historical_credit_card_agg.join(
    historical_credit_card.group_by("SK_ID_PREV").agg(pl.count().alias("NUM_ENTRIES")),
    on="SK_ID_PREV",
)

over-limit aggs:

In [7]:
historical_credit_card_agg = eda.make_aggregations(
    historical_credit_card_agg,
    historical_credit_card,
    ["IS_OVER_LIMIT", "OVER_LIMIT"],
    "SK_ID_PREV",
)
historical_credit_card_agg = historical_credit_card_agg.drop(
    columns=["IS_OVER_LIMIT_min", "IS_OVER_LIMIT_max"]
)

In [8]:
active_credit_cards = active_credit_cards.join(
    historical_credit_card_agg.select(
        [pl.col("SK_ID_PREV"), pl.all().exclude("SK_ID_PREV").prefix("HISTORIC_")]
    ),
    on="SK_ID_PREV",
    how="left",
)

### Application Attributes

In [9]:
credit_card_applications = previous_application.filter(
    pl.col("SK_ID_PREV").is_in(active_credit_cards["SK_ID_PREV"])
)

**Dropping features irrelevant to credit cards:**

In [10]:
cols_to_drop = [
    "SK_ID_CURR",
    "NAME_CONTRACT_TYPE",
    "AMT_DOWN_PAYMENT",
    "AMT_GOODS_PRICE",
    "RATE_DOWN_PAYMENT",
    "RATE_DOWN_PAYMENT",
    "RATE_INTEREST_PRIMARY",
    "RATE_INTEREST_PRIVILEGED",
    "NAME_CASH_LOAN_PURPOSE",
    "NAME_CONTRACT_STATUS",
    "DAYS_DECISION",
    "NAME_PAYMENT_TYPE",
    "FLAG_LAST_APPL_PER_CONTRACT",
    "CODE_REJECT_REASON",
    "NAME_GOODS_CATEGORY",
    "NAME_PORTFOLIO",
    "CNT_PAYMENT",
    "NAME_YIELD_GROUP",
    "DAYS_FIRST_DRAWING",
    "DAYS_FIRST_DUE",
    "DAYS_LAST_DUE_1ST_VERSION",
    "DAYS_LAST_DUE",
    "DAYS_TERMINATION",
]
credit_card_applications = credit_card_applications.drop(columns=cols_to_drop)

In [11]:
active_credit_cards=active_credit_cards.join(credit_card_applications.select(
        [pl.col("SK_ID_PREV"), pl.all().exclude("SK_ID_PREV").prefix("APPLICATION_")]
    ),on='SK_ID_PREV',how="left")

### Payment Behavioral Data

In [12]:
credit_card_installments = installments_payments.filter(
    pl.col("SK_ID_PREV").is_in(active_credit_cards["SK_ID_PREV"])
)
# Filtering out new data to avoid leakage
credit_card_installments = credit_card_installments.filter(
    (pl.col("DAYS_INSTALMENT") < -31) & (pl.col("DAYS_ENTRY_PAYMENT") < -31)
)

In [13]:
installments_agg = pl.DataFrame(active_credit_cards["SK_ID_PREV"])

In [14]:
credit_card_installments = credit_card_installments.with_columns(
    (pl.col("DAYS_INSTALMENT") - pl.col("DAYS_ENTRY_PAYMENT")).alias("DAYS_LATE")
)

credit_card_installments = credit_card_installments.with_columns(
    pl.when(pl.col("DAYS_LATE") < 0)
    .then(pl.lit(0))
    .otherwise(pl.col("DAYS_LATE"))
    .alias("DAYS_LATE")
)


credit_card_installments = credit_card_installments.with_columns(
    (pl.col("AMT_INSTALMENT") - pl.col("AMT_PAYMENT")).alias("AMT_UNDERPAID")
)
credit_card_installments = credit_card_installments.with_columns(
    pl.when(pl.col("AMT_UNDERPAID") < 0)
    .then(pl.lit(0))
    .otherwise(pl.col("AMT_UNDERPAID"))
    .alias("AMT_UNDERPAID")
)

In [15]:
installments_agg = eda.make_aggregations(
    installments_agg,
    credit_card_installments,
    ["DAYS_LATE"],
    id="SK_ID_PREV",
    aggregations=["mean", "sum", "max"],
    join_suffix="inst"
)

installments_agg = eda.make_aggregations(
    installments_agg,
    credit_card_installments,
    ["AMT_UNDERPAID"],
    id="SK_ID_PREV",
    aggregations=["mean", "sum", "max"],
    join_suffix="inst"
)

In [16]:
active_credit_cards = active_credit_cards.join(
    installments_agg.select(
        [pl.col("SK_ID_PREV"), pl.all().exclude("SK_ID_PREV").prefix("instalment_")]
    ),
    on="SK_ID_PREV",
    how="left",
)


### Current status

In [17]:
application_cols_to_merge = [
    "SK_ID_CURR",
    "CODE_GENDER",
    "FLAG_OWN_REALTY",
    "CNT_CHILDREN",
    "AMT_INCOME_TOTAL",
    "NAME_TYPE_SUITE",
    "NAME_INCOME_TYPE",
    "NAME_EDUCATION_TYPE",
    "NAME_FAMILY_STATUS",
    "NAME_HOUSING_TYPE",
    "REGION_POPULATION_RELATIVE",
    "DAYS_BIRTH",
    "DAYS_EMPLOYED",
    "OWN_CAR_AGE",
    "OCCUPATION_TYPE",
    "CNT_FAM_MEMBERS",
    "REGION_RATING_CLIENT",
    "REGION_RATING_CLIENT_W_CITY",
    "REG_REGION_NOT_LIVE_REGION",
    "REG_REGION_NOT_WORK_REGION",
    "LIVE_REGION_NOT_WORK_REGION",
    "REG_CITY_NOT_LIVE_CITY",
    "REG_CITY_NOT_WORK_CITY",
    "LIVE_CITY_NOT_WORK_CITY",
    "ORGANIZATION_TYPE",
]
active_credit_cards=active_credit_cards.join(application_full[application_cols_to_merge],on="SK_ID_CURR",how='left')

In [18]:
active_credit_cards=active_credit_cards.drop("SK_ID_PREV","SK_ID_CURR")

## Feature Selection

### Variance

In [19]:
str_cols=active_credit_cards.select(pl.col(pl.Utf8)).columns

In [20]:
home_target_encoders = {}
active_credit_cards_encoded = active_credit_cards.clone()
for col in str_cols:
    home_target_encoders[col] = tr.TargetMeanOrderedLabeler(how="label")
    active_credit_cards_encoded = active_credit_cards_encoded.with_columns(
        home_target_encoders[col]
        .fit_transform(
            active_credit_cards_encoded[col], active_credit_cards_encoded["IS_OVER_LIMIT"]
        )
        .alias(col)
    )
active_credit_cards_encoded_scaled = pl.DataFrame(
    MinMaxScaler().fit_transform(active_credit_cards_encoded),
    schema=active_credit_cards_encoded.columns,
)

In [21]:
feature_variance = pl.DataFrame(
    {
        "feature": active_credit_cards_encoded_scaled.columns,
        "variance": np.var(
            active_credit_cards_encoded_scaled.to_numpy(), axis=0
        ),
    }
)
feature_variance.sort("variance")[:20]

feature,variance
str,f64
"""SK_DPD_DEF""",0.000045
"""AMT_INCOME_TOT…",0.000067
"""AMT_PAYMENT_TO…",0.000445
"""AMT_DRAWINGS_C…",0.000465
"""SK_DPD""",0.000796
"""CNT_CHILDREN""",0.001457
"""CNT_DRAWINGS_C…",0.001846
"""CNT_FAM_MEMBER…",0.00221
"""REG_REGION_NOT…",0.012806


In [22]:
feature_variance.filter(pl.col("variance") < 0.001)["feature"]

feature
str
"""AMT_DRAWINGS_C…"
"""AMT_PAYMENT_TO…"
"""SK_DPD"""
"""SK_DPD_DEF"""
"""AMT_INCOME_TOT…"


In [23]:
low_var_fatures = []
for feature in feature_variance.filter(pl.col("variance") < 0.001)["feature"]:
    if active_credit_cards_encoded_scaled[feature].n_unique() < 10:
        low_var_fatures.append(feature)
        minority_count = (
            active_credit_cards_encoded_scaled[feature]
            .value_counts()
            .sort("counts")["counts"][0]
        )
        print(
            f"{feature}: minority class count = {minority_count} feature will be removed"
        )

### Highly Correlated

In [24]:
feature_correlation_test = st.get_correlation_pairs(
    active_credit_cards_encoded.drop(columns="IS_OVER_LIMIT"),
    max_threshold=0.999,
    min_threshold=-0.999,
)
print(tabulate(feature_correlation_test['clusters']))

-------------  --------------------
AMT_RECIVABLE  AMT_TOTAL_RECEIVABLE
-------------  --------------------


In [25]:
feature_correlation_test['pairs']

features,correlation
list[str],f64
"[""AMT_RECIVABLE"", ""AMT_TOTAL_RECEIVABLE""]",1.0


In [26]:
active_credit_cards=active_credit_cards.drop(columns="AMT_TOTAL_RECEIVABLE")

In [27]:
active_credit_cards.write_parquet('temp/active_credit_cards.parquet')

## Model 3

In [28]:
consumer_loans = previous_application.filter(
    (pl.col("NAME_CONTRACT_TYPE") == "Consumer loans")
    & (pl.col("NAME_CONTRACT_STATUS") == "Approved")
)

# Irrelevant cols
consumer_loans = consumer_loans.drop(
    columns=[
        "NAME_YIELD_GROUP",
        "NAME_CONTRACT_STATUS",
        "NAME_CONTRACT_TYPE",
        "NAME_CASH_LOAN_PURPOSE",
        "NAME_CASH_LOAN_PURPOSE",
        "NAME_PRODUCT_TYPE",
        "DAYS_FIRST_DRAWING",
        "DAYS_FIRST_DRAWING",
        "DAYS_FIRST_DUE",
        "DAYS_LAST_DUE_1ST_VERSION",
        "DAYS_LAST_DUE",
        "DAYS_TERMINATION",
        "SK_ID_PREV",
        "SK_ID_CURR"
    ]
)

consumer_loans = consumer_loans.with_columns(
    (
        (pl.col("AMT_ANNUITY") * pl.col("CNT_PAYMENT") - pl.col("AMT_CREDIT"))
        / pl.col("AMT_CREDIT")
        * 100
    ).alias("FEES_PERCENT")
)

In [29]:
consumer_loans.head()

AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,FLAG_LAST_APPL_PER_CONTRACT,NFLAG_LAST_APPL_IN_DAY,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,DAYS_DECISION,NAME_PAYMENT_TYPE,CODE_REJECT_REASON,NAME_TYPE_SUITE,NAME_CLIENT_TYPE,NAME_GOODS_CATEGORY,NAME_PORTFOLIO,CHANNEL_TYPE,SELLERPLACE_AREA,NAME_SELLER_INDUSTRY,CNT_PAYMENT,PRODUCT_COMBINATION,NFLAG_INSURED_ON_APPROVAL,FEES_PERCENT
f64,f64,f64,f64,f64,str,i64,str,i64,f64,f64,f64,i64,str,str,str,str,str,str,str,i64,str,f64,str,f64,f64
1730.43,17145.0,17145.0,0.0,17145.0,"""SATURDAY""",15,"""Y""",1,0.0,0.182832,0.867336,-73,"""Cash through t…","""XAP""",null,"""Repeater""","""Mobile""","""POS""","""Country-wide""",35,"""Connectivity""",12.0,"""POS mobile wit…",0.0,21.114961
7654.86,53779.5,57564.0,0.0,53779.5,"""SUNDAY""",15,"""Y""",1,0.0,null,null,-408,"""Cash through t…","""XAP""","""Unaccompanied""","""New""","""Consumer Elect…","""POS""","""Country-wide""",200,"""Consumer elect…",8.0,"""POS household …",1.0,6.38399
9644.22,26550.0,27252.0,0.0,26550.0,"""SATURDAY""",10,"""Y""",1,0.0,null,null,-726,"""Cash through t…","""XAP""",null,"""New""","""Construction M…","""POS""","""Stone""",83,"""Consumer elect…",3.0,"""POS household …",0.0,6.167107
21307.455,126490.5,119853.0,12649.5,126490.5,"""TUESDAY""",7,"""Y""",1,0.103971,null,null,-699,"""Cash through t…","""XAP""","""Unaccompanied""","""New""","""Auto Accessori…","""POS""","""Regional / Loc…",130,"""Industry""",6.0,"""POS other with…",0.0,6.667943
4187.34,26955.0,27297.0,1350.0,26955.0,"""SATURDAY""",12,"""Y""",1,0.051324,null,null,-1473,"""Cash through t…","""XAP""","""Unaccompanied""","""Repeater""","""Photo / Cinema…","""POS""","""Stone""",130,"""Consumer elect…",8.0,"""POS household …",0.0,22.71942


### Variance

In [30]:
str_cols_consumer=consumer_loans.select(pl.col(pl.Utf8)).columns

In [31]:
consumer_encoders = {}
consumer_loans_encoded = consumer_loans.clone()
for col in str_cols_consumer:
    consumer_encoders[col] = tr.TargetMeanOrderedLabeler(how="label")
    consumer_loans_encoded = consumer_loans_encoded.with_columns(
        consumer_encoders[col]
        .fit_transform(
            consumer_loans_encoded[col], consumer_loans_encoded["FEES_PERCENT"]
        )
        .alias(col)
    )
consumer_loans_encoded_scaled = pl.DataFrame(
    MinMaxScaler().fit_transform(consumer_loans_encoded),
    schema=consumer_loans_encoded.columns,
)

In [32]:
feature_variance_consumer = pl.DataFrame(
    {
        "feature": consumer_loans_encoded_scaled.columns,
        "variance": np.var(
            consumer_loans_encoded_scaled.to_numpy(), axis=0
        ),
    }
)
feature_variance_consumer.sort("variance")[:20]

feature,variance
str,f64
"""FLAG_LAST_APPL…",0.0
"""SELLERPLACE_AR…",0.000008
"""CODE_REJECT_RE…",0.000013
"""AMT_APPLICATIO…",0.000253
"""AMT_GOODS_PRIC…",0.000253
"""AMT_CREDIT""",0.000381
"""NAME_PORTFOLIO…",0.00042
"""NFLAG_LAST_APP…",0.000426
"""CNT_PAYMENT""",0.006346


In [33]:
feature_variance_consumer.filter(pl.col("variance") < 0.001)["feature"]

feature
str
"""AMT_APPLICATIO…"
"""AMT_CREDIT"""
"""AMT_GOODS_PRIC…"
"""FLAG_LAST_APPL…"
"""NFLAG_LAST_APP…"
"""CODE_REJECT_RE…"
"""NAME_PORTFOLIO…"
"""SELLERPLACE_AR…"


In [34]:
low_var_fatures_consumer = []
for feature in feature_variance_consumer.filter(pl.col("variance") < 0.001)["feature"]:
    if consumer_loans_encoded_scaled[feature].n_unique() < 10:
        low_var_fatures_consumer.append(feature)
        minority_count = (
            consumer_loans_encoded_scaled[feature]
            .value_counts()
            .sort("counts")["counts"][0]
        )
        print(
            f"{feature}: minority class count = {minority_count} feature will be removed"
        )

FLAG_LAST_APPL_PER_CONTRACT: minority class count = 626470 feature will be removed
NFLAG_LAST_APPL_IN_DAY: minority class count = 267 feature will be removed
CODE_REJECT_REASON: minority class count = 8 feature will be removed
NAME_PORTFOLIO: minority class count = 263 feature will be removed


In [35]:
consumer_loans = consumer_loans.drop(columns=low_var_fatures_consumer)

consumer_loans_encoded = consumer_loans_encoded.drop(
    columns=low_var_fatures_consumer
)

### Highly Correlated

In [36]:
feature_correlation_test_consumer = st.get_correlation_pairs(
    consumer_loans_encoded.drop(columns="FEES_PERCENT"),
    max_threshold=0.999,
    min_threshold=-0.999,
)
feature_correlation_test_consumer['pairs']

features,correlation
list[str],f64
"[""AMT_APPLICATION"", ""AMT_GOODS_PRICE""]",1.0


In [37]:
consumer_loans=consumer_loans.drop(columns="AMT_GOODS_PRICE")
consumer_loans.write_parquet("temp/consumer_loans.parquet")